In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append("..")

import numpy as np
import skfda
from sklearn.model_selection import train_test_split

from fundl.config import FunDLnBlockConfig, FunDLObjective
from fundl.model import FunDL

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


### Load Data

### Process Data for FNN

In [3]:
fda_data, obs_data = skfda.datasets.fetch_tecator(return_X_y=True, as_frame=False)
data = np.concatenate(
    (
        fda_data.data_matrix,
        fda_data.derivative().data_matrix,
        fda_data.derivative().derivative().data_matrix,
    ),
    axis=2,
)

X_train, X_val, obs_data_train, obs_data_val = train_test_split(
    data, obs_data, test_size=0.2, random_state=42
)
y_train = obs_data_train[:,0]
y_val = obs_data_val[:,0]

### Build Model

In [6]:
import tensorflow as tf

hidden_layers = [
    FunDLnBlockConfig(
        name="01_relu_dropout_16",
        units=16,
        activation="relu",
        dropout_rate=None,
    ),
    FunDLnBlockConfig(
        name="02_relu_dropout_16",
        units=32,
        activation="relu",
        dropout_rate=None,
    ),
    FunDLnBlockConfig(
        name="03_relu_dropout_16",
        units=16,
        activation="relu",
        dropout_rate=None,
    ),
    FunDLnBlockConfig(
        name="04_relu_16",
        units=8,
        activation="relu",
        add_dropout=False,
        dropout_rate=None,
    ),
]
input_shape = 100
output_size = 1
model = FunDL(
    objective=FunDLObjective("classification"),
    input_shape=input_shape,
    hidden_layers=hidden_layers,
    output_size=output_size,
)
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001, decay = 0),
    loss="mse", 
    metrics=["mean_squared_error"]

)
_ = model.summary(line_length=100, show_trainable=True)

Model: "FunDL"
_______________________________________________________________________________________________________________
 Layer (type)                                Output Shape                            Param #        Trainable  
 input (InputLayer)                          [(None, 100, 3)]                        0              Y          
                                                                                                               
 01_relu_dropout_16 (DenseDropoutBlock)      (None, 100, 16)                         64             Y          
                                                                                                               
 02_relu_dropout_16 (DenseDropoutBlock)      (None, 100, 32)                         544            Y          
                                                                                                               
 03_relu_dropout_16 (DenseDropoutBlock)      (None, 100, 16)                         528 

In [11]:
_ = model.fit(X_train, y_train, batch_size=16, epochs=50, verbose=1)

Epoch 1/50
11/11 [==============================] - 0s 2ms/step - loss: 449.1162 - mean_squared_error: 449.1162
Epoch 2/50
11/11 [==============================] - 0s 2ms/step - loss: 449.1164 - mean_squared_error: 449.1165
Epoch 3/50
11/11 [==============================] - 0s 2ms/step - loss: 449.1164 - mean_squared_error: 449.1163
Epoch 4/50
11/11 [==============================] - 0s 2ms/step - loss: 449.1163 - mean_squared_error: 449.1163
Epoch 5/50
11/11 [==============================] - 0s 2ms/step - loss: 449.1163 - mean_squared_error: 449.1162
Epoch 6/50
11/11 [==============================] - 0s 2ms/step - loss: 449.1164 - mean_squared_error: 449.1164
Epoch 7/50
11/11 [==============================] - 0s 2ms/step - loss: 449.1161 - mean_squared_error: 449.1162
Epoch 8/50
11/11 [==============================] - 0s 2ms/step - loss: 449.1165 - mean_squared_error: 449.1165
Epoch 9/50
11/11 [==============================] - 0s 2ms/step - loss: 449.1161 - mean_squared_error: 4

In [9]:
import tensorflow as tf
print(model.fit.__doc__)

Trains the model for a fixed number of epochs (iterations on a dataset).

    Args:
        x: Input data. It could be:
          - A Numpy array (or array-like), or a list of arrays
            (in case the model has multiple inputs).
          - A TensorFlow tensor, or a list of tensors
            (in case the model has multiple inputs).
          - A dict mapping input names to the corresponding array/tensors,
            if the model has named inputs.
          - A `tf.data` dataset. Should return a tuple
            of either `(inputs, targets)` or
            `(inputs, targets, sample_weights)`.
          - A generator or `keras.utils.Sequence` returning `(inputs, targets)`
            or `(inputs, targets, sample_weights)`.
          - A `tf.keras.utils.experimental.DatasetCreator`, which wraps a
            callable that takes a single argument of type
            `tf.distribute.InputContext`, and returns a `tf.data.Dataset`.
            `DatasetCreator` should be used when us

In [18]:
df_func.derivative().derivative().data_matrix

array([[[ 1.65392206e-05],
        [ 1.77643480e-05],
        [ 2.38899853e-05],
        ...,
        [-2.08271667e-05],
        [-9.80101960e-06],
        [ 9.80101960e-06]],

       [[-2.14397304e-05],
        [-3.06281863e-06],
        [ 1.59266569e-05],
        ...,
        [-2.29711397e-04],
        [-2.04596284e-04],
        [-1.72742970e-04]],

       [[-1.40889657e-05],
        [-6.12563725e-07],
        [ 1.34764020e-05],
        ...,
        [ 1.28025819e-04],
        [ 1.10261471e-04],
        [ 9.55599411e-05]],

       ...,

       [[ 3.49161323e-05],
        [ 3.36910049e-05],
        [ 3.36910049e-05],
        ...,
        [-1.65392206e-04],
        [-1.38439402e-04],
        [-1.01685578e-04]],

       [[ 6.12563725e-05],
        [ 4.16543333e-05],
        [ 3.24658774e-05],
        ...,
        [-2.60339583e-04],
        [-2.34611907e-04],
        [-2.06433975e-04]],

       [[ 2.63402402e-05],
        [ 4.22668970e-05],
        [ 6.18689362e-05],
        ...,
        